# Getting the required code and models

In [ ]:
!git clone https://github.com/FreshMag/omr-img2midi.git
%cd omr-img2midi
!mkdir models
%cd models
!wget https://grfia.dlsi.ua.es/primus/models/PrIMuS/Semantic-Model.zip
!unzip Semantic-Model.zip -d semantic